# Notebook para cargar los datos

In [1]:
import pandas as pd
import numpy as np

## Datos de CIQ

In [2]:
file = "data/2023.xlsx"
df = pd.read_excel(file)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,Fecha,45291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Excel Company ID,Rating,Sector,Current Assets,Cash and equivalents,Receivables,Total Assets,Current Liabilities,Long Term Debt,Total Debt,Total Equity,Gastos financieros netos,EBITDA,Company Name
3,NaN,IQ12316552,#PEND,Financials,0,0,0,0,0,0,0,0,0,NaN,0
4,NaN,IQ79178550,NR,Financials,130.214214,77.455044,20.555678,408.599825,285.338413,98.692132,99.292631,23.436533,0,NaN,0


In [3]:
columns_to_check = ["Current Assets", "Cash and equivalents", "Receivables", "Total Assets", "Current Liabilities", "Long Term Debt", "Total Debt", "Total Equity", "Gastos financieros netos", "EBITDA"]
df = pd.read_excel(file, skiprows=3, usecols="B:O")
mask = (df[columns_to_check] != 0) & (df[columns_to_check] != "NaN")
df = df[mask.any(axis=1)]
df.head()


,Excel Company ID,Rating,Sector,Current Assets,Cash and equivalents,Receivables,Total Assets,Current Liabilities,Long Term Debt,Total Debt,Total Equity,Gastos financieros netos,EBITDA,Company Name
0,IQ12316552,#PEND,Financials,0,0,0,0,0,0,0,0,0,NaN,0
1,IQ79178550,NR,Financials,130.214214,77.455044,20.555678,408.599825,285.338413,98.692132,99.292631,23.436533,0,NaN,0
11,IQ4533704,NR,Financials,0,9.997955,0,1254.853235,0,0,0,89.331279,0,0,0
12,IQ321778,NR,Consumer Discretionary,2097.53549,390.09665,23.5214,4553.895707,1841.580941,1058.086658,2738.44571,154.571102,-57.429495,190.14611,OTCPK:BBBY.Q
24,IQ627329,BBB,Utilities,686.505,46.038,614.501,5603.582,802.918,3086.998,3230.616,1300.308,-57.47,496.025,0


In [4]:
name_of_columns = ["Excel Company ID","Rating","Sector", "Company Name"] + columns_to_check

In [5]:
# Check if all the files have the same columns

for year in range(2014, 2024):
    file = f"data/{year}.xlsx"
    if year >= 2019:
        df = pd.read_excel(file, skiprows=3, usecols="B:O")
    else:
        df = pd.read_excel(file, skiprows=3, usecols="B:Y")
    df = df[name_of_columns]
    print(year,set(df.columns) - set(name_of_columns))


2014 set()
2015 set()
2016 set()
2017 set()
2018 set()
2019 set()
2020 set()
2021 set()
2022 set()
2023 set()


In [6]:
# Now we will do it for all the excel
columns_to_check = ["Current Assets", "Cash and equivalents", "Receivables", "Total Assets", "Current Liabilities", "Long Term Debt", "Total Debt", "Total Equity", "Gastos financieros netos", "EBITDA"]

for year in range(2014, 2024):
    file = f"data/{year}.xlsx"
    if year >= 2019:
        df = pd.read_excel(file, skiprows=3, usecols="B:O")
    else:
        df = pd.read_excel(file, skiprows=3, usecols="B:Y")
        
    df = df[name_of_columns]
    mask = (df[columns_to_check] != 0) & (df[columns_to_check] != "NaN")
    df = df[mask.any(axis=1)]
    df['year'] = year
    # merge all the df
    if year == 2014:
        df_all = df
    else:
        df_all = pd.concat([df_all, df])
    print(f"{year} done") 

2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done
2022 done
2023 done


In [7]:
# create the new columns with the ratios more importants
for columnas in name_of_columns[3:]:
    df_all[columnas] = pd.to_numeric(df_all[columnas], errors='coerce')

In [8]:
df_all[name_of_columns] == "#REFRESH"
# count the number of rows that have "#REFRESH" in any of the columns
df_all[name_of_columns].apply(lambda x: x == "#REFRESH").sum(axis=0)

Excel Company ID            0
Rating                      1
Sector                      0
Company Name                0
Current Assets              0
Cash and equivalents        0
Receivables                 0
Total Assets                0
Current Liabilities         0
Long Term Debt              0
Total Debt                  0
Total Equity                0
Gastos financieros netos    0
EBITDA                      0
dtype: int64

In [9]:

df_all["Net Debt to EBITDA"] = (df_all["Total Debt"] - df_all["Cash and equivalents"]) / df_all["EBITDA"]
df_all["Debt to Assets"] = df_all["Total Debt"] / (df_all["Total Equity"]+df_all["Total Debt"])
df_all["Long Term Debt to Equity"] = df_all["Long Term Debt"] / df_all["Total Equity"]
df_all["Financial Leverage"] = df_all["Total Assets"] / df_all["Total Equity"]

df_all["Current Ratio"] = df_all["Current Assets"] / df_all["Current Liabilities"]
df_all["Quick Ratio"] = (df_all["Cash and equivalents"] + df_all["Receivables"]) / df_all["Current Liabilities"]
df_all["Cash ratio"] = df_all["Cash and equivalents"] / df_all["Current Liabilities"]
df_all["Debt to Equity"] = df_all["Total Debt"] / df_all["Total Equity"]
df_all["Debt ratio"] = df_all["Total Assets"] / df_all["Total Debt"]

In [10]:
df_all.to_csv("all_data.csv", index=False)